In [1]:
!pip install opencv-python mediapipe pyautogui


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (

  DEPRECATION: Building 'pygetwindow' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pygetwindow'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'pytweening' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pytweening'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'mouseinfo' using the legacy setup.py bdist_wh

In [2]:
import cv2
import mediapipe as mp
import pyautogui
from time import sleep


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)


In [4]:
tip_ids = [4, 8, 12, 16, 20]

def count_fingers(hand_landmarks):
    fingers = []

    # Thumb
    if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # Other four fingers
    for id in range(1, 5):
        if hand_landmarks.landmark[tip_ids[id]].y < hand_landmarks.landmark[tip_ids[id] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers


In [5]:
cap = cv2.VideoCapture(0)

prev_action = None  # to avoid repeating same command

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for mirror effect
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            fingers = count_fingers(hand_landmarks)
            total_fingers = fingers.count(1)

            # Map to media actions
            action = None
            if total_fingers == 1:
                action = 'play_pause'
            elif total_fingers == 2:
                action = 'next'
            elif total_fingers == 3:
                action = 'previous'
            elif total_fingers == 4:
                action = 'volume_up'
            elif total_fingers == 5:
                action = 'volume_down'

            # Trigger action if it's new
            if action != prev_action:
                if action == 'play_pause':
                    pyautogui.press('space')
                elif action == 'next':
                    pyautogui.hotkey('ctrl', 'right')
                elif action == 'previous':
                    pyautogui.hotkey('ctrl', 'left')
                elif action == 'volume_up':
                    pyautogui.press('volumeup')
                elif action == 'volume_down':
                    pyautogui.press('volumedown')
                prev_action = action
                sleep(1)  # prevent multiple triggers

            # Show current action on screen
            if action:
                cv2.putText(frame, f'Action: {action}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 255, 0), 2)

    cv2.imshow("Smart Gesture Media Controller", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
